In [ ]:
import requests#maps with message 
import time
import folium
import openrouteservice as ors
from twilio.rest import Client

# Twilio Credentials (same as your original script)
TWILIO_ACCOUNT_SID_1 = 'ACc0fd62adb6fedfd78dd600ad5c3d1da1'
TWILIO_AUTH_TOKEN_1 = '767d6ce6efd44c1249c1a7d069328b41'
TWILIO_PHONE_NUMBER_1 = '+12315454495'
USER_PHONE_NUMBER_1 = "+919676490994"

TWILIO_ACCOUNT_SID_2 = 'AC131241e4ad648945771bc618ca08ae0c'
TWILIO_AUTH_TOKEN_2 = '1a4d915cb8d147c3ce500de91e138c09'
TWILIO_PHONE_NUMBER_2 = '+12315998788'
USER_PHONE_NUMBER_2 = "+917794027054"

TWILIO_ACCOUNT_SID_3 = 'AC6b5302179768a0bdccd74f2b3a12b1f0'
TWILIO_AUTH_TOKEN_3 = '75a6fe6073b6e55307d2584805e03c03'
TWILIO_PHONE_NUMBER_3 = '+12314409690'
USER_PHONE_NUMBER_3 = "+918801764966"

# ThingSpeak Channel Details (same as in your original script)
channel_1_id = "2784770"
read_api_1_key = "I6I38IODBRXVZCAV"

channel_2_id = "2795362"
read_api_2_key = "225NJ2MKZB1U4RH0"

channel_3_id = "2795423"
read_api_3_key = "ZZJLS0LI3ZUS7W84"

# Set to keep track of sent alerts
sent_alerts = set()

# Function to fetch the last value from ThingSpeak for all fields
def fetch_last_values(channel_id, api_key, field_count):
    url = f"https://api.thingspeak.com/channels/{channel_id}/feeds.json"
    params = {"api_key": api_key, "results": 1}
    response = requests.get(url, params=params)

    if response.status_code == 200:
        data = response.json()
        if data and "feeds" in data and len(data["feeds"]) > 0:
            last_entry = data["feeds"][-1]
            sensor_data = {}
            for field_num in range(1, field_count + 1):
                sensor_data[f"field{field_num}"] = last_entry.get(f"field{field_num}")
            return sensor_data
        else:
            print("No data found in the specified fields.")
            return None
    else:
        print(f"Failed to fetch data: {response.status_code}")
        return None

# Function to check the alert conditions
def check_alert_conditions(sensor_data):
    if sensor_data:
        field_6 = float(sensor_data.get("field6", 0))
        field_7 = float(sensor_data.get("field7", 0))
        field_8 = float(sensor_data.get("field8", 0))

        if field_6 < 5 and field_7 == 0 and field_8 == 0:
            return True
    return False

# Function to send SMS alert using Twilio
def send_sms_alert(account_sid, auth_token, from_number, to_number, message):
    try:
        client = Client(account_sid, auth_token)
        message = client.messages.create(
            body=message,
            from_=from_number,
            to=to_number
        )
        print(f"SMS sent: {message.sid}")
        return True
    except Exception as e:
        print(f"Error sending SMS: {e}")
        return False

# Function to update the map based on the alert condition
def update_map(is_alert_triggered):
    # Create a map centered at the source location (Beeramguda)
    source = [78.3033, 17.5019]
    destination = [78.4138, 17.4949]
    m = folium.Map(location=list(reversed(source)), tiles="cartodbpositron", zoom_start=14)

    # Add markers for source and destination
    folium.Marker(location=list(reversed(source)), icon=folium.Icon(color="red", icon="cloud"), popup="Beeramguda").add_to(m)
    folium.Marker(location=list(reversed(destination)), icon=folium.Icon(color="blue", icon="flag"), popup="Kukatpally").add_to(m)

    # Initialize the OpenRouteService client
    client = ors.Client(key='5b3ce3597851110001cf6248f71e4da02d6f409bb1bc92a59f0d107e')

    # Request two alternative routes between the source and destination
    routes = client.directions(
        coordinates=[source, destination],
        profile='driving-car',
        alternative_routes={"share_factor": 0.5, "target_count": 2},  # Request two alternative routes
        geometry=True
    )

    # Extract and represent the main route in red
    main_route_coords = ors.convert.decode_polyline(routes['routes'][0]['geometry'])
    folium.PolyLine(locations=[list(reversed(coord)) for coord in main_route_coords['coordinates']], color='red', weight=5, popup="Main Route").add_to(m)

    # If an alert is triggered, display the second optimized route
    if is_alert_triggered and len(routes['routes']) > 1:
        alternative_route_coords = ors.convert.decode_polyline(routes['routes'][1]['geometry'])
        folium.PolyLine(locations=[list(reversed(coord)) for coord in alternative_route_coords['coordinates']], color='blue', weight=5, popup="Alternative Route").add_to(m)

    # Return the updated map
    return m

# Main loop for fetching data and checking alerts
initial_display = True  # Flag to display the first route initially
while True:
    # Fetch the latest sensor data for all nodes
    sensor_data_node1 = fetch_last_values(channel_1_id, read_api_1_key, 8)
    sensor_data_node2 = fetch_last_values(channel_2_id, read_api_2_key, 8)
    sensor_data_node3 = fetch_last_values(channel_3_id, read_api_3_key, 8)

    # Check alert conditions for each node
    alert_node1 = check_alert_conditions(sensor_data_node1)
    alert_node2 = check_alert_conditions(sensor_data_node2)
    alert_node3 = check_alert_conditions(sensor_data_node3)

    # Handle alerts and send SMS if necessary
    if alert_node1 and alert_node2 and not alert_node3:
        alert_key = ("Node1", "Node2")
        if alert_key not in sent_alerts:
            send_sms_alert(TWILIO_ACCOUNT_SID_3, TWILIO_AUTH_TOKEN_3, TWILIO_PHONE_NUMBER_3, USER_PHONE_NUMBER_3, "Accident occurred at Node 1 and Node 2!")
            sent_alerts.add(alert_key)

    elif alert_node2 and alert_node3 and not alert_node1:
        alert_key = ("Node2", "Node3")
        if alert_key not in sent_alerts:
            send_sms_alert(TWILIO_ACCOUNT_SID_1, TWILIO_AUTH_TOKEN_1, TWILIO_PHONE_NUMBER_1, USER_PHONE_NUMBER_1, "Accident occurred at Node 2 and Node 3!")
            sent_alerts.add(alert_key)

    elif alert_node1 and alert_node3 and not alert_node2:
        alert_key = ("Node1", "Node3")
        if alert_key not in sent_alerts:
            send_sms_alert(TWILIO_ACCOUNT_SID_2, TWILIO_AUTH_TOKEN_2, TWILIO_PHONE_NUMBER_2, USER_PHONE_NUMBER_2, "Accident occurred at Node 1 and Node 3!")
            sent_alerts.add(alert_key)

    elif alert_node1:
        alert_key = ("Node1",)
        if alert_key not in sent_alerts:
            send_sms_alert(TWILIO_ACCOUNT_SID_2, TWILIO_AUTH_TOKEN_2, TWILIO_PHONE_NUMBER_2, USER_PHONE_NUMBER_2, "Accident occurred at Node 1!")
            send_sms_alert(TWILIO_ACCOUNT_SID_3, TWILIO_AUTH_TOKEN_3, TWILIO_PHONE_NUMBER_3, USER_PHONE_NUMBER_3, "Accident occurred at Node 1!")
            sent_alerts.add(alert_key)

    elif alert_node2:
        alert_key = ("Node2",)
        if alert_key not in sent_alerts:
            send_sms_alert(TWILIO_ACCOUNT_SID_1, TWILIO_AUTH_TOKEN_1, TWILIO_PHONE_NUMBER_1, USER_PHONE_NUMBER_1, "Accident occurred at Node 2!")
            send_sms_alert(TWILIO_ACCOUNT_SID_3, TWILIO_AUTH_TOKEN_3, TWILIO_PHONE_NUMBER_3, USER_PHONE_NUMBER_3, "Accident occurred at Node 2!")
            sent_alerts.add(alert_key)

    elif alert_node3:
        alert_key = ("Node3",)
        if alert_key not in sent_alerts:
            send_sms_alert(TWILIO_ACCOUNT_SID_1, TWILIO_AUTH_TOKEN_1, TWILIO_PHONE_NUMBER_1, USER_PHONE_NUMBER_1, "Accident occurred at Node 3!")
            send_sms_alert(TWILIO_ACCOUNT_SID_2, TWILIO_AUTH_TOKEN_2, TWILIO_PHONE_NUMBER_2, USER_PHONE_NUMBER_2, "Accident occurred at Node 3!")
            sent_alerts.add(alert_key)

    # Initially display the main route, then update map based on alert conditions
    if initial_display:
        m = update_map(is_alert_triggered=False)  # Show main route initially
        initial_display = False  # Set flag to False after first display
    else:
        # Display map based on current alert status
        is_alert_triggered = alert_node1 or alert_node2 or alert_node3
        m = update_map(is_alert_triggered)

    # Display the map (works in Jupyter notebook, adjust display method as needed)
    display(m)

    # Pause for a while before the next update
    time.sleep(5)
